In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
url_re = re.compile("https?://[^\s]+")
mention_re = re.compile("@(\w+)")
def preprocessing(text):
    text_clean = url_re.sub("<url>",text)
    text_clean = mention_re.sub("<mention>", text_clean)
    text_clean = text_clean.lower()
    return text_clean 

In [ ]:
def representationBOW(corpus):
    vectorizerTrain = TfidfVectorizer(ngram_range = (3,5),max_df=0.95,min_df=2,analyzer="char_wb")
    bow = vectorizerTrain.fit_transform(corpus)
    return bow,vectorizerTrain

In [ ]:
import csv
dataText = []
dataLabels = []
i = 0
with open('Tweets.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    for row in csv_reader:
        if i >0:
            dataText.append(preprocessing(row[10]))
            dataLabels.append(row[1])
        i+=1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataText, dataLabels, test_size=0.20, random_state=42)

In [ ]:
bowTrain,vectorizer = representationBOW(X_train,character=True)

In [ ]:
bowTest = vectorizer.transform(X_test)

In [ ]:
from sklearn.svm import SVC
kernels = ["rbf","linear"]
cs = [1,10,100,1000]
for kernel in kernels:
    for c in cs:
        clf = SVC(kernel=kernel,C=c)
        clf.fit(bowTrain, y_train)
        print("%s \t %d \t %.3f " %(kernel[0:3],c,clf.score(bowTest,y_test)))

In [ ]:
from sklearn.linear_model import LogisticRegression
cs = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 10, 100, 1000]
for c in cs:
    clf = LogisticRegression(C=c)
    clf.fit(bowTrain, y_train)
    print("%.3f \t %.3f " %(c,clf.score(bowTest,y_test)))